In [5]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Path to the dataset
path = './dataset/train/'
class_names = sorted(os.listdir(path))
num_classes = len(class_names)

# Image size
img_size = (192, 192)  # EfficientNetB0 input size

# Load and preprocess data
labels = []
images = []

for cl in class_names:
    for img in os.listdir(path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        labels.append(label)
        
        image = cv2.imread(path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        images.append(image)

# Convert to numpy arrays
labels = np.asarray(labels)
images = np.asarray(images)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.15, random_state=42)

# Load EfficientNetB0 pre-trained on ImageNet without top layer
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=img_size + (3,))

# Add custom top layers for skin diseases classification
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint('efficientnet_model/model{epoch:02d}')


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 192, 192, 3)]        0         []                            
                                                                                                  
 rescaling_2 (Rescaling)     (None, 192, 192, 3)          0         ['input_2[0][0]']             
                                                                                                  
 normalization_1 (Normaliza  (None, 192, 192, 3)          7         ['rescaling_2[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_3 (Rescaling)     (None, 192, 192, 3)          0         ['normalization_1[0][0]'

In [6]:
# Train the model

from tensorflow.keras.callbacks import EarlyStopping

custom_early_stopping = EarlyStopping(
    monitor='val_accuracy',
    #baseline=0.80,  # Set the baseline accuracy threshold
    patience=3,
    mode='max',
    verbose=1
)

    
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[custom_early_stopping])

Epoch 1/20
179/179 [==============================] - 106s 534ms/step - loss: 1.6402 - accuracy: 0.4186 - val_loss: 1.5198 - val_accuracy: 0.4758
Epoch 2/20
179/179 [==============================] - 100s 559ms/step - loss: 1.3108 - accuracy: 0.5437 - val_loss: 1.4292 - val_accuracy: 0.5124
Epoch 3/20
179/179 [==============================] - 162s 903ms/step - loss: 1.1151 - accuracy: 0.6212 - val_loss: 1.3947 - val_accuracy: 0.5153
Epoch 4/20
179/179 [==============================] - 167s 936ms/step - loss: 0.9478 - accuracy: 0.6750 - val_loss: 1.3805 - val_accuracy: 0.5124
Epoch 5/20
179/179 [==============================] - 170s 950ms/step - loss: 0.8022 - accuracy: 0.7293 - val_loss: 1.4645 - val_accuracy: 0.5262
Epoch 6/20
179/179 [==============================] - 170s 951ms/step - loss: 0.6759 - accuracy: 0.7730 - val_loss: 1.4160 - val_accuracy: 0.5371
Epoch 7/20
179/179 [==============================] - 169s 946ms/step - loss: 0.5644 - accuracy: 0.8112 - val_loss: 1.5122 -

In [17]:
test_path = 'dataset/test1/'

# Load and preprocess test data
test_labels = []
test_images = []

class_names = sorted(os.listdir(test_path))
num_classes = len(class_names)

for cl in class_names:
    for img in os.listdir(test_path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        test_labels.append(label)
        
        image = cv2.imread(test_path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        test_images.append(image)

# Convert to numpy arrays
test_labels = np.asarray(test_labels)
test_images = np.asarray(test_images)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

54/54 [==============================] - 84s 1s/step - loss: 1.5604 - accuracy: 0.5919
Test Loss: 1.5604
Test Accuracy: 0.5919


In [8]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

# Path to the dataset
path = './dataset/train/'
class_names = sorted(os.listdir(path))
num_classes = len(class_names)

# Image size
img_size = (192, 192)  # EfficientNetB0 input size

# Load and preprocess data
labels = []
images = []

for cl in class_names:
    for img in os.listdir(path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        labels.append(label)
        
        image = cv2.imread(path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        images.append(image)

# Convert to numpy arrays
labels = np.asarray(labels)
images = np.asarray(images)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.15, random_state=42)

# Load EfficientNetB0 pre-trained on ImageNet without top layer
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=img_size + (3,))

# Add custom top layers for skin diseases classification with L2 regularization
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint('efficientnet_model/model{epoch:02d}')

# EarlyStopping callback
custom_early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    mode='max',
    verbose=1
)
    
# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[checkpoint_callback, custom_early_stopping])


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 192, 192, 3)]        0         []                            
                                                                                                  
 rescaling_4 (Rescaling)     (None, 192, 192, 3)          0         ['input_3[0][0]']             
                                                                                                  
 normalization_2 (Normaliza  (None, 192, 192, 3)          7         ['rescaling_4[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_5 (Rescaling)     (None, 192, 192, 3)          0         ['normalization_2[0][0]'

INFO:tensorflow:Assets written to: efficientnet_model\model01\assets


179/179 [==============================] - 125s 642ms/step - loss: 3.7841 - accuracy: 0.4181 - val_loss: 2.3006 - val_accuracy: 0.4688
Epoch 2/20
179/179 [==============================] - ETA: 0s - loss: 1.9882 - accuracy: 0.4904INFO:tensorflow:Assets written to: efficientnet_model\model02\assets


INFO:tensorflow:Assets written to: efficientnet_model\model02\assets


179/179 [==============================] - 112s 625ms/step - loss: 1.9882 - accuracy: 0.4904 - val_loss: 1.9869 - val_accuracy: 0.4322
Epoch 3/20
179/179 [==============================] - ETA: 0s - loss: 1.7658 - accuracy: 0.5086INFO:tensorflow:Assets written to: efficientnet_model\model03\assets


...

179/179 [==============================] - 115s 643ms/step - loss: 1.4951 - accuracy: 0.5905 - val_loss: 1.7434 - val_accuracy: 0.5094
Epoch 11/20
179/179 [==============================] - ETA: 0s - loss: 1.4855 - accuracy: 0.5915INFO:tensorflow:Assets written to: efficientnet_model\model11\assets


INFO:tensorflow:Assets written to: efficientnet_model\model11\assets


179/179 [==============================] - 114s 637ms/step - loss: 1.4855 - accuracy: 0.5915 - val_loss: 1.7355 - val_accuracy: 0.4936
Epoch 12/20
179/179 [==============================] - ETA: 0s - loss: 1.4587 - accuracy: 0.6041INFO:tensorflow:Assets written to: efficientnet_model\model12\assets


INFO:tensorflow:Assets written to: efficientnet_model\model12\assets


179/179 [==============================] - 115s 644ms/step - loss: 1.4587 - accuracy: 0.6041 - val_loss: 1.7621 - val_accuracy: 0.4807
Epoch 12: early stopping


In [9]:
test_path = 'dataset/test/'

# Load and preprocess test data
test_labels = []
test_images = []

class_names = sorted(os.listdir(test_path))
num_classes = len(class_names)

for cl in class_names:
    for img in os.listdir(test_path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        test_labels.append(label)
        
        image = cv2.imread(test_path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        test_images.append(image)

# Convert to numpy arrays
test_labels = np.asarray(test_labels)
test_images = np.asarray(test_images)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

56/56 [==============================] - 23s 414ms/step - loss: 1.7050 - accuracy: 0.5237
Test Loss: 1.7050
Test Accuracy: 0.5237


In [11]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

# Path to the dataset
path = './dataset/train/'
class_names = sorted(os.listdir(path))
num_classes = len(class_names)

# Image size
img_size = (192, 192)  # EfficientNetB0 input size

# Load and preprocess data
labels = []
images = []

for cl in class_names:
    for img in os.listdir(path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        labels.append(label)
        
        image = cv2.imread(path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        images.append(image)

# Convert to numpy arrays
labels = np.asarray(labels)
images = np.asarray(images)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.15, random_state=42)

# Load EfficientNetB0 pre-trained on ImageNet without top layer
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=img_size + (3,))

# Add custom top layers for skin diseases classification with L2 regularization
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint('efficientnet_model/model{epoch:02d}')

# EarlyStopping callback
custom_early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    mode='max',
    verbose=1
)
    
# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[checkpoint_callback, custom_early_stopping])


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 192, 192, 3)]        0         []                            
                                                                                                  
 rescaling_8 (Rescaling)     (None, 192, 192, 3)          0         ['input_5[0][0]']             
                                                                                                  
 normalization_4 (Normaliza  (None, 192, 192, 3)          7         ['rescaling_8[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_9 (Rescaling)     (None, 192, 192, 3)          0         ['normalization_4[0][0]'

INFO:tensorflow:Assets written to: efficientnet_model\model01\assets


179/179 [==============================] - 657s 3s/step - loss: 3.1410 - accuracy: 0.4237 - val_loss: 2.2803 - val_accuracy: 0.4491
Epoch 2/20
179/179 [==============================] - ETA: 0s - loss: 1.9815 - accuracy: 0.4913INFO:tensorflow:Assets written to: efficientnet_model\model02\assets


INFO:tensorflow:Assets written to: efficientnet_model\model02\assets


179/179 [==============================] - 306s 2s/step - loss: 1.9815 - accuracy: 0.4913 - val_loss: 1.9698 - val_accuracy: 0.4520
Epoch 3/20
179/179 [==============================] - ETA: 0s - loss: 1.7643 - accuracy: 0.5150INFO:tensorflow:Assets written to: efficientnet_model\model03\assets


...

179/179 [==============================] - 718s 4s/step - loss: 1.4637 - accuracy: 0.6053 - val_loss: 1.7406 - val_accuracy: 0.5163
Epoch 13/20
179/179 [==============================] - ETA: 0s - loss: 1.4568 - accuracy: 0.6069INFO:tensorflow:Assets written to: efficientnet_model\model13\assets


INFO:tensorflow:Assets written to: efficientnet_model\model13\assets


179/179 [==============================] - 948s 5s/step - loss: 1.4568 - accuracy: 0.6069 - val_loss: 1.7411 - val_accuracy: 0.5035
Epoch 14/20
179/179 [==============================] - ETA: 0s - loss: 1.4438 - accuracy: 0.6109INFO:tensorflow:Assets written to: efficientnet_model\model14\assets


INFO:tensorflow:Assets written to: efficientnet_model\model14\assets


179/179 [==============================] - 791s 4s/step - loss: 1.4438 - accuracy: 0.6109 - val_loss: 1.7607 - val_accuracy: 0.4857
Epoch 14: early stopping


In [13]:
test_path = 'dataset/test1/'

# Load and preprocess test data
test_labels = []
test_images = []

class_names = sorted(os.listdir(test_path))
num_classes = len(class_names)

for cl in class_names:
    for img in os.listdir(test_path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        test_labels.append(label)
        
        image = cv2.imread(test_path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        test_images.append(image)

# Convert to numpy arrays
test_labels = np.asarray(test_labels)
test_images = np.asarray(test_images)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

67/67 [==============================] - 90s 1s/step - loss: 1.6073 - accuracy: 0.5648
Test Loss: 1.6073
Test Accuracy: 0.5648
